In [1]:
import json
import os
import pandas as pd
import pickle, dill

from flask import Flask
from flask import jsonify
from flask import render_template
from flask import request
from flask import url_for
# from googleapiclient import discovery
# from oauth2client.client import GoogleCredentials
from xgboost import XGBClassifier
from category_encoders import MEstimateEncoder


class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=4)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    
with open('TEncoder.pickle', 'rb') as handle:
    tencoder = pickle.load(handle)

te_features = ['C1', 'OWN', 1]
# te_features = [data['subgrade'], data['home_ownership'], 1]
te_features = pd.DataFrame(te_features).T
te_features.columns = ['sub_grade', 'home_ownership', 'target']
encoded_features = tencoder.transform(te_features)

print(encoded_features.sub_grade_encoded)
print(pd.__version__)
# print(pickle.__version__)
# !pip freeze

0    0.186802
Name: sub_grade_encoded, dtype: float64
1.3.5


In [2]:
!pip freeze

absl-py==1.4.0
aiohttp==3.8.3
aiohttp-cors==0.7.0
aiorwlock==1.3.0
aiosignal==1.3.1
alembic==1.10.2
ansiwrap==0.8.4
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1666191106763/work/dist
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1649500320262/work
async-timeout==4.0.2
asynctest==0.13.0
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1671632566681/work
Babel==2.11.0
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backoff==1.10.0
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beatrix-jupyterlab @ file:///home/kbuilder/miniconda3/conda-bld/dlenv-base_1674528954702/work/beatrix_jupyterlab-2023.123.173907.tar.gz
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup